In [1]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

Because the best results so far were achieved using a combined dataset with feature engineering and a Random Forest model, it allows to think that tree-based models may be the most optimal for this problem.

Because if that XGBoost was selected for further testing.

### Preprocessing

In [3]:
proteins_with_add = pd.read_csv("./Data/additional_features_dataset.csv")

In [4]:
proteins_with_add

,Unnamed: 0,structureId,chainId,sequence,residueCount,len,A,C,D,E,...,chainCount,resolution,structureMolecularWeight,crystallizationTempK,densityMatthews,densityPercentSol,phValue,beg_motif,end_motif,proteins_encoded
0,0,101M,A,MVLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDR...,154,154,0.110390,0.000000,0.038961,0.090909,...,1,2.07,18112.80,291.037497,3.09,60.20,9.000000,26,-1,4
1,1,102L,A,MNIFEMLRIDEGLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAAKSE...,165,165,0.103030,0.000000,0.060606,0.048485,...,1,1.74,18926.61,291.174328,2.75,55.28,6.662035,6,-1,0
2,2,102M,A,MVLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDR...,154,154,0.116883,0.000000,0.038961,0.090909,...,1,1.84,18010.64,291.037497,3.09,60.20,9.000000,26,-1,4
3,3,103L,A,MNIFEMLRIDEGLRLKIYKDTEGYYTIGIGHLLTKSPSLNSLDAAK...,167,167,0.095808,0.000000,0.065868,0.047904,...,1,1.90,19092.72,291.174328,2.70,54.46,6.662035,6,-1,0
4,4,103M,A,MVLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDR...,154,154,0.116883,0.000000,0.038961,0.090909,...,1,2.07,18093.78,291.037497,3.09,60.30,9.000000,26,-1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117508,117508,9RNT,A,ACDYTCGSNCYSSSDVSTAQAAGYKLHEDGETVGSNSYPHKYNNYE...,104,104,0.067308,0.038462,0.057692,0.057692,...,1,1.50,11134.77,291.174328,2.09,41.23,6.662035,-1,-1,0
117509,117509,9RSA,A,KETAAAKFERQHMDSSTSAASSSNYCNQMMKSRNLTKDRCKPVNTF...,248,124,0.096774,0.064516,0.040323,0.040323,...,2,1.80,27987.16,291.174328,2.25,45.45,6.662035,14,-1,0
117510,117510,9RUB,A,MDQSSRYVNLALKEEDLIAGGEHVLCAYIMKPKAGYGYVATAAHFA...,932,466,0.133047,0.010730,0.062232,0.057940,...,2,2.60,101838.68,290.894503,2.38,48.29,7.082046,-1,-1,5
117511,117511,9XIA,A,MNYQPTPEDRFTFGLWTVGWQGRDPFGDATRRALDPVESVQRLAEL...,388,388,0.121134,0.002577,0.095361,0.072165,...,1,1.90,43542.29,291.368018,2.79,55.93,6.840613,-1,-1,9


In [5]:
proteins_with_add = proteins_with_add.drop_duplicates(subset="structureId",keep="first").reset_index()

In [6]:
proteins_with_add.drop(columns=["Unnamed: 0" , "index"],inplace=True)

In [7]:
proteins_with_add

,structureId,chainId,sequence,residueCount,len,A,C,D,E,F,...,chainCount,resolution,structureMolecularWeight,crystallizationTempK,densityMatthews,densityPercentSol,phValue,beg_motif,end_motif,proteins_encoded
0,101M,A,MVLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDR...,154,154,0.110390,0.000000,0.038961,0.090909,0.032468,...,1,2.07,18112.80,291.037497,3.09,60.20,9.000000,26,-1,4
1,102L,A,MNIFEMLRIDEGLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAAKSE...,165,165,0.103030,0.000000,0.060606,0.048485,0.030303,...,1,1.74,18926.61,291.174328,2.75,55.28,6.662035,6,-1,0
2,102M,A,MVLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDR...,154,154,0.116883,0.000000,0.038961,0.090909,0.038961,...,1,1.84,18010.64,291.037497,3.09,60.20,9.000000,26,-1,4
3,103L,A,MNIFEMLRIDEGLRLKIYKDTEGYYTIGIGHLLTKSPSLNSLDAAK...,167,167,0.095808,0.000000,0.065868,0.047904,0.029940,...,1,1.90,19092.72,291.174328,2.70,54.46,6.662035,6,-1,0
4,103M,A,MVLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDR...,154,154,0.116883,0.000000,0.038961,0.090909,0.038961,...,1,2.07,18093.78,291.037497,3.09,60.30,9.000000,26,-1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117508,9RNT,A,ACDYTCGSNCYSSSDVSTAQAAGYKLHEDGETVGSNSYPHKYNNYE...,104,104,0.067308,0.038462,0.057692,0.057692,0.038462,...,1,1.50,11134.77,291.174328,2.09,41.23,6.662035,-1,-1,0
117509,9RSA,A,KETAAAKFERQHMDSSTSAASSSNYCNQMMKSRNLTKDRCKPVNTF...,248,124,0.096774,0.064516,0.040323,0.040323,0.024194,...,2,1.80,27987.16,291.174328,2.25,45.45,6.662035,14,-1,0
117510,9RUB,A,MDQSSRYVNLALKEEDLIAGGEHVLCAYIMKPKAGYGYVATAAHFA...,932,466,0.133047,0.010730,0.062232,0.057940,0.049356,...,2,2.60,101838.68,290.894503,2.38,48.29,7.082046,-1,-1,5
117511,9XIA,A,MNYQPTPEDRFTFGLWTVGWQGRDPFGDATRRALDPVESVQRLAEL...,388,388,0.121134,0.002577,0.095361,0.072165,0.059278,...,1,1.90,43542.29,291.368018,2.79,55.93,6.840613,-1,-1,9


### Model

In [8]:
from xgboost import XGBClassifier

In [ ]:
df_size = len(proteins_with_add)
train_size = int(len(proteins_with_add)*0.7)

proteins_cleaned_cleaned = proteins_with_add.sample(frac=1)

train = proteins_cleaned_cleaned[:train_size]
test = proteins_cleaned_cleaned[train_size:]

X_train = train.drop(columns=['structureId','chainId','sequence','classification',"proteins_encoded"])
y_train = train["proteins_encoded"]

X_test = test.drop(columns=['structureId','chainId','sequence','classification',"proteins_encoded"])
y_test = test["proteins_encoded"]

In [14]:
from sklearn.utils.class_weight import compute_sample_weight

sample_weights = compute_sample_weight(class_weight='balanced', y=y_train)

In [39]:
def test_result(model, X_test, y_test):
    y_pred = model.predict(X_test)
    print(f"accuracy: {accuracy_score(y_test, y_pred)}")
    print(f"F1: {f1_score(y_test, y_pred, average='macro')}")
    print(f"precision: {precision_score(y_test, y_pred, average='macro')}")
    print(f"recall: {recall_score(y_test, y_pred, average='macro')}")

In [23]:
from xgboost import XGBClassifier

model = XGBClassifier(
    booster='gbtree',
    n_estimators=1000,
    max_depth=6,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    num_parallel_tree=1,
    tree_method='hist',
    random_state=42
)

model.fit(X_train, y_train, sample_weight = sample_weights)


XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.1, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=1000,
              n_jobs=None, num_parallel_tree=1, ...)

In [25]:
test_result(model, X_test, y_test)

accuracy: 0.8494638906223407
F1: 0.8286681572377274
precision: 0.8672108300122695
recall: 0.7983445093789425


In [ ]:
from xgboost import XGBClassifier

model = XGBClassifier(
    booster='gbtree',
    n_estimators=1000,
    max_depth=10,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    num_parallel_tree=1,
    tree_method='hist',
    random_state=42
)

model.fit(X_train, y_train, sample_weight = sample_weights)

Grid search was performed to find the optimal hyperparameter values: n_estimators parameter was fixed at 100, as lower values were unlikely to obtain satisfactory results, while higher values significantly increased execution time.

In [31]:
from sklearn.model_selection import GridSearchCV

params = {
    'n_estimators': [100],
    'max_depth': [5, 10],
    'learning_rate': [0.1, 0.01],
    'subsample': [0.5, 1]
}

model = XGBClassifier(booster='gbtree',
                      tree_method='hist',
                      random_state=42)

grid_search = GridSearchCV(model, params, cv=5, scoring='accuracy')

grid_search.fit(X_train, y_train, sample_weight = sample_weights)

lprint(f"Best params: {grid_search.best_params_}, acc: {grid_search.best_score_}")

Best params: {'learning_rate': 0.1, 'max_depth': 10, 'n_estimators': 100, 'subsample': 1}, acc: 0.7994991573159456


Based on the best model 3 more tests were performed:
1. With same number of estimators but higher depth (15)
2. With increased number estimators (1000) but same depth (10)
3. With increased number of estimators and increased depth 

In [38]:
model = XGBClassifier(
    booster='gbtree',
    n_estimators=100,
    max_depth=15,
    learning_rate=0.1,
    subsample=1,
    num_parallel_tree=1,
    tree_method='hist',
    random_state=42
)

model.fit(X_train, y_train, sample_weight = sample_weights)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.1, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=15,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=100,
              n_jobs=None, num_parallel_tree=1, ...)

In [40]:
test_result(model, X_test, y_test)

accuracy: 0.8343450388608384
F1: 0.8083029639157175
precision: 0.8355499916357414
recall: 0.7871782364170741


In [34]:
model = XGBClassifier(
    booster='gbtree',
    n_estimators=1000,
    max_depth=10,
    learning_rate=0.1,
    subsample=1,
    num_parallel_tree=1,
    tree_method='hist',
    random_state=42
)

model.fit(X_train, y_train, sample_weight = sample_weights)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.1, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=10,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=1000,
              n_jobs=None, num_parallel_tree=1, ...)

In [35]:
test_result(model, X_test, y_test)

accuracy: 0.8527259318091565
F1: 0.8266620182039722
precision: 0.8628336182605472
recall: 0.797428307239633


In [46]:
model = XGBClassifier(
    booster='gbtree',
    n_estimators=1000,
    max_depth=15,
    learning_rate=0.1,
    subsample=1,
    num_parallel_tree=1,
    tree_method='hist',
    random_state=42
)

model.fit(X_train, y_train, sample_weight = sample_weights)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.1, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=15,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=1000,
              n_jobs=None, num_parallel_tree=1, ...)

In [47]:
test_result(model, X_test, y_test)

accuracy: 0.8517047710898054
F1: 0.8226111602247141
precision: 0.8543132529892498
recall: 0.7968177182288617


The final test showed no improvement, so the optimal hyperparameters remained:

- n_estimators=1000,
- max_depth=10,
- learning_rate=0.1,
- subsample=1,
- num_parallel_tree=1

With a result of: 
- accuracy: 0.85
- F1: 0.83
- precision: 0.86
- recall: 0.80